In [1]:
from setup_llm import load_llm, embed_model
from llama_index.node_parser import CodeSplitter
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage, get_response_synthesizer,StorageContext, ServiceContext
from llama_index.indices.composability import ComposableGraph

In [2]:
def get_code_parser(language="go"):
    return CodeSplitter.from_defaults(
        language=language,
    )
    
def get_auto_merging_ctx_code(llm, language="go"):
    return ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=get_code_parser(language=language),
    )

In [3]:
llm = load_llm()
code_automerging_idx = get_auto_merging_ctx_code(llm)

automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./preproc_data/fiber_merging_index_c"),
        service_context=code_automerging_idx,
    )

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
summary = automerging_index.query(
    "What is a summary of this document?", retriever_mode="all_leaf"
)
index1_summary = str(summary)

AttributeError: 'VectorStoreIndex' object has no attribute 'query'

In [5]:
automerging_retriver = automerging_index.as_retriever(
    similarity_top_k=12,
)

retriever = AutoMergingRetriever(
    automerging_retriver,
    automerging_index.storage_context,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(service_context=code_automerging_idx, verbose=True)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

In [6]:
query_engine = RetrieverQueryEngine(
    retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[rerank],
)

In [8]:
response = query_engine.query(
    "What is a summary of this document?"
)

from llama_index.response.notebook_utils import display_response
display_response(response)

/home/alfagov/anaconda3/envs/rag/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/alfagov/anaconda3/envs/rag/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The document describes two separate components: a file system explorer built using HTML rendering and integration with the Fiber framework, and a custom context implementation for a web application framework with Fasthttp and Go Testing Framework. The file system explorer allows for the discovery of unique entities, titles, and themes through directory listing, file browsing, and error handling. The custom context implementation includes features such as user context setting, cookie handling, request and response access, and unit tests for content negotiation and query parameter prefixing. The document also touches on HTTP request handling and Go testing, as well as benchmarking.

In [16]:
from llama_index.chat_engine.condense_question import (
    CondenseQuestionChatEngine,
)

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    llm=llm,
    verbose=True,
)

In [17]:
resp = chat_engine.chat("What are the setters available for widgets that diapley numeric values?")
display_response(resp)

/home/alfagov/anaconda3/envs/rag/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/alfagov/anaconda3/envs/rag/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Querying with: In the context of discussing widgets that display numeric values, what setters are available to customize their display?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** In the context of discussing widgets that display numeric values, the available setters to customize their display include:

1. SetNumericUpdateHandler: This setter allows you to provide a function that returns the updated value and error for the numeric widget. This function is called when the widget needs to update its value, and it should return the new value and any errors that occurred during the update.

2. SetNumericInitValue: This setter allows you to provide an integer value as the initial value for the numeric widget. This value is displayed when the widget is first loaded.

3. SetNumericUnit: This setter allows you to provide a string representation of the measurement unit to be displayed alongside the numeric value. This unit helps to provide context and make the value more meaningful to the user.

4. SetNumericUnitAfter: This setter allows you to toggle the position of the unit to follow the numeric value. By default, the unit is displayed after the value, but you can set this setter to `true` to display the unit before the value.

These setters can be used to customize the behavior and appearance of numeric widgets, such as

In [18]:
resp = chat_engine.chat("Which one defines the logic for updating the value displayed by the widget?")
display_response(resp)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Querying with: What setter is responsible for defining the logic for updating the value displayed by widgets that display numeric values?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The setter responsible for defining the logic for updating the value displayed by widgets that display numeric values using the NumericWidget function is the `setUpdateHandler` method in the `numeric` struct. This method takes a handler function that returns an integer value and an error as arguments, and assigns it to the `updateHandler` field of the `numeric` struct. This handler function is called when the widget needs to update its value, and it should return the new value and any errors that occur during the update process. By customizing the `updateHandler` function, developers can provide unique features and preferences for displaying numerical data, such as dynamic updates, custom themes, and layout options provided by goDashboard's Layout package. This integration enhances the user experience and makes the data more intuitive and user-friendly.